In [ ]:
# Import packages needed & load image
import gc
import pickle
from tkinter import Tk
from tkinter.filedialog import askopenfilename, asksaveasfilename
import platform


from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib.widgets import RectangleSelector, PolygonSelector
import numpy as np
from skimage import io, draw
import mplcursors
import IPython.display as Disp
from ipywidgets import widgets
import cv2
from osgeo import gdal
import pandas as pd

import general_funcs

# OS related settings
if platform.system() == 'Windows':
    # %matplotlib nbagg
    # Sometimes tk/qt will not let cells rerun after an ERROR occurs
    # %matplotlib tk
    %matplotlib qt
elif platform.system() == 'Darwin':
    Tk().withdraw()
    %matplotlib osx
elif platform == 'linux' or platform == 'linux2':
    ;
# This line of "print" must exist right after %matplotlib command, 
# otherwise JN will hang on the first import statement after this.
print('Interactive plot activated')


# Use skimage to load multi-layer tiff file
# Tk().withdraw()
image_file = askopenfilename(title='Load image file', initialdir='./data/field_image')
img = io.imread(image_file)
print('Image File:', image_file)
print("Image Shape:", img.shape)

h, w, d = img.shape

# Extract layers from the multilayer tiff file and do some adjustments
layer_RGB, layer_IR, layer_mask = general_funcs.extract_layers(img)

del(img)
# Play sound when done
general_funcs.done()

In [ ]:
# Select MTPs

def onselect(vert):
    ;
    
def set_mtps(mtp):
    def on_click(event):
        mtp.append([event.xdata, event.ydata])
        ax.text(event.xdata, event.ydata, 'MTP ' + str(len(mtp)))

        circle = patches.Circle([event.xdata, event.ydata], radius=50, color='yellow')
        ax.add_patch(circle)
    return(on_click)


def reset_mtps(mtp):
    def on_key(event):
        if event.key == 'escape':
            for t in ax.texts:
                t.set_visible(False)
            ax.texts.clear()
            ax.patches.clear()
            mtp.clear()
    return(on_key)

fig, ax = plt.subplots(figsize=(7, 7))
plt.subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95)
if d == 5 or d == 4:
    myax = ax.imshow(layer_RGB)
elif d == 2:
    mask_not_0_inds = np.where(layer_mask > 0)
    vmin, vmax = general_funcs.cal_vmin_vmax(layer_IR, layer_mask)
    myax = ax.imshow(layer_IR, cmap='gist_gray', vmin=vmin, vmax=vmax)
    cbar = fig.colorbar(myax)
    
ps = PolygonSelector(ax, onselect, useblit=True, lineprops=dict(linewidth=0))

mtp = []

cid_set_mtps = fig.canvas.mpl_connect('button_press_event', set_mtps(mtp))
cid_reset_mtps = fig.canvas.mpl_connect('key_press_event', reset_mtps(mtp))